# Recommend Customer's most expensive purchases in the hope that she will buy it again
Well the the Header is self explanatory. One nuance is that if the customer has made < 12 purchases, he will be recommended from another list of overall most expensive items
For example, 
* if a customer X has purchased 20 items, most expensive 12 of her purchases will be recommended again.
* if a customer Y has purchased 5 items, those 5 AND 7 of the overall most expensive 7 across all customers will be recommended.
* if a customer Z has purchased 0 items, 12 of the overall most expensive 7 across all customers will be recommended.



In [ ]:
import pandas as pd


**Load Transactions**


In [ ]:
train = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv')
train['article_id'] = train.article_id.astype('int32')
train.t_dat = pd.to_datetime(train.t_dat)
train = train[['t_dat','customer_id','article_id','price']]
train.to_parquet('train.pqt',index=False)
print( train.shape )
train.head()

**Get the most expensive purchase of each customer**

In [ ]:
trans_costliest_first = train.sort_values(['customer_id','price'],ascending=False)
trans_costliest_first['article_id'] = '0' + trans_costliest_first['article_id'].astype(str)
trans_costliest_first['article_id'] = trans_costliest_first['article_id'].astype(str)
trans_costliest_first = trans_costliest_first.drop_duplicates(subset=['customer_id','article_id'],keep='last')
trans_costliest_first.to_parquet('trans_costliest_first.pqt')
trans_costliest_first.head(5)

In [ ]:
trans_top_buys_only = trans_costliest_first[['customer_id','article_id']]

**Get Overall most expensive**

In [ ]:
top12 = trans_costliest_first.sort_values('price',ascending=False).drop_duplicates().head(12)
top12 = top12['article_id']
top12 = ' '.join(list(top12))
top12 = [top12]
top12

In [ ]:
purchase_dict = {}

for i,x in enumerate(zip(trans_top_buys_only['customer_id'], trans_top_buys_only['article_id'])):
    cust_id, art_id = x
    if cust_id not in purchase_dict:
        purchase_dict[cust_id] = {}
    
    if art_id not in purchase_dict[cust_id]:
        purchase_dict[cust_id][art_id] = 0
    
    purchase_dict[cust_id][art_id] += 1
    


In [ ]:
submission = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/sample_submission.csv')
submission =  submission.drop('prediction',axis=1)
#submission

In [ ]:
simplest = submission[['customer_id']]
#simplest= simplest.to_pandas()
prediction_list=[]
for i, cust_id in enumerate(submission['customer_id'].values.reshape((-1,))):
    if cust_id in purchase_dict:
        l = sorted((purchase_dict[cust_id]).items(), key=lambda x: x[1], reverse=True)
        l = [y[0] for y in l]
        if len(l)>12:
            s = ' '.join(l[:12])
        else:
            s = ' '.join(l+top12[:(12-len(l))])
    else:
        s = top12
    prediction_list.append(s)
    
simplest['prediction'] = prediction_list
#print(simplest.shape)
#simplest.head()

In [ ]:
simplest

In [ ]:
simplest.to_csv(f'submission.csv',index=False)
simplest.head()


# Credits:
* Some ideas and code is from Chris Deotte's excellent https://www.kaggle.com/code/cdeotte/recommend-items-purchased-together-0-021
* Some ideas and code is also from Abhilash Awasthis' https://www.kaggle.com/code/abhilashawasthi/not-so-fancy-but-fast-benchmark 